In [1]:
from collections import Counter

from xgboost import XGBRegressor
from sklearn.model_selection import RepeatedKFold, cross_val_score
import pandas as pd
import numpy as np

/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
with open('auto-mpg.csv', 'r') as fp:
    df = pd.read_csv(fp, na_values='?')

# Data Analysis

In [3]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger


<b>Size of the dataset is quite small, and neural networks tend to overfit on small datasets. While a small MLP would perhaps fare ok, I prefer using XGBoost due to the tabular nature of the data.</b>

## Idea #1: Create an extra column from the car's manufacturer

We are going to assume that the word of the `car name` column is the manufacturer. This will give us another datapoint. While the physical characteristics like weight and horsepower should be much more indicative of the value of the target variable, one could assume that each manufacturer has their own proprietary technology that might reduce the mpg or something along those lines. Under the mantra of **_more data cannot hurt_**, we will perform an with and without these added feature.

In [4]:
df_mfct = df.copy()
df_mfct['mfct'] = df_mfct['car name'].transform(lambda x: x.split(' ')[0])
df_mfct

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,mfct
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,buick
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,plymouth
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,amc
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,ford
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl,ford
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup,vw
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage,dodge
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger,ford


In [5]:
Counter(df_mfct.mfct), f'NUMBER OF MANUFACTURERS: {len(set(Counter(df_mfct.mfct)))}'

(Counter({'chevrolet': 43,
          'buick': 17,
          'plymouth': 31,
          'amc': 28,
          'ford': 51,
          'pontiac': 16,
          'dodge': 28,
          'toyota': 25,
          'datsun': 23,
          'volkswagen': 15,
          'peugeot': 8,
          'audi': 7,
          'saab': 4,
          'bmw': 2,
          'chevy': 3,
          'hi': 1,
          'mercury': 11,
          'opel': 4,
          'fiat': 8,
          'oldsmobile': 10,
          'chrysler': 6,
          'mazda': 10,
          'volvo': 6,
          'renault': 5,
          'toyouta': 1,
          'maxda': 2,
          'honda': 13,
          'subaru': 4,
          'chevroelt': 1,
          'capri': 1,
          'vw': 6,
          'mercedes-benz': 2,
          'cadillac': 2,
          'mercedes': 1,
          'vokswagen': 1,
          'triumph': 1,
          'nissan': 1}),
 'NUMBER OF MANUFACTURERS: 37')

Our assumption seems to hold pretty well! Although there is some minor data noise: `volkswagen` and `vw` likely refer to the same manufacturer. Same goes with `mercedes-benz` and `mercedes` or `maxda` and `mazda` (cute typo!) or `toyota` and `toyouta`. Let's quickly solve this issue.

In [6]:
replacement_dict = {
    'vw': 'volkswagen',
    'vokswagen': 'volkswagen',
    'mercedes': 'mercedes-benz',
    'maxda': 'mazda',
    'toyouta': 'toyota',
    'chevroelt': 'chevrolet',
    'chevy': 'chevrolet',
    'capri': 'ford'
}
df_mfct['mfct'] = df_mfct['mfct'].transform(lambda x: replacement_dict[x] if x in replacement_dict else x)
Counter(df_mfct.mfct)

Counter({'chevrolet': 47,
         'buick': 17,
         'plymouth': 31,
         'amc': 28,
         'ford': 52,
         'pontiac': 16,
         'dodge': 28,
         'toyota': 26,
         'datsun': 23,
         'volkswagen': 22,
         'peugeot': 8,
         'audi': 7,
         'saab': 4,
         'bmw': 2,
         'hi': 1,
         'mercury': 11,
         'opel': 4,
         'fiat': 8,
         'oldsmobile': 10,
         'chrysler': 6,
         'mazda': 12,
         'volvo': 6,
         'renault': 5,
         'honda': 13,
         'subaru': 4,
         'mercedes-benz': 3,
         'cadillac': 2,
         'triumph': 1,
         'nissan': 1})

I will quickly drop exactly 1 row for which I cannot find any extra information out in the wild. It is for the greater good and won't hurt us too much.

In [7]:
df_mfct = df_mfct[df_mfct['mfct'] != 'hi']

## Idea #2: Use the geographical position of the producer as another data point

I will admit, this might inch towards overthinking. But different manufacturers design cars for different consumer realities e.g. USA based producers might not have fuel efficiency i.e. mpg in mind when building a car since gasoline is quite cheap in the country. Let us introduce a new categorical feature for where the manufacturer is located. On the other hand, big manufacturers tend to ship worlwide + the dataset seems to be compiled on USA-based cars from Kaggle's dataset metadata. This feature might be thoroughly useless, but investigating is worth it. We will look the data on the internet for this task.

<p style="color: red; font-weight: bold">Nota bene: The origin feature might be doing this already. It is categorically encoded, with a domain of [1, 2, 3]. These might be the USA, Europe, Asia i.e. continents of origin, but the metadata says nothign about this. We will proceed with both, although some duplication might be possible.</p>

In [8]:
geo_dict = {
    'chevrolet': 'USA',
    'buick': 'USA',
    'plymouth': 'USA',
    'amc': 'USA',
    'ford': 'USA',
    'pontiac': 'USA',
    'dodge': 'USA',
    'toyota': 'Japan',
    'datsun': 'Japan',
    'volkswagen': 'Germany',
    'peugeot': 'France',
    'audi': 'Germany',
    'saab': 'Sweden',
    'bmw': 'Germany',
    'mercury': 'USA',
    'opel': 'Germany',
    'fiat': 'Italy',
    'oldsmobile': 'USA',
    'chrysler': 'USA',
    'mazda': 'Japan',
    'volvo': 'Sweden',
    'renault': 'France',
    'honda': 'Japan',
    'subaru': 'Japan',
    'mercedes-benz': 'Germany',
    'cadillac': 'USA',
    'triumph': 'UK',
    'nissan': 'Japan'
}

Pretty nice! Most companies are clustered around `USA`, `Japan` and `Germany`, which raises my hopes

In [9]:
df_mfct_geo = df_mfct.copy()
df_mfct_geo['geo'] = df_mfct_geo['mfct'].apply(lambda x: geo_dict[x].lower())
df_mfct_geo

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name,mfct,geo
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,chevrolet,usa
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,buick,usa
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,plymouth,usa
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,amc,usa
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,ford,usa
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1,ford mustang gl,ford,usa
394,44.0,4,97.0,52.0,2130,24.6,82,2,vw pickup,volkswagen,germany
395,32.0,4,135.0,84.0,2295,11.6,82,1,dodge rampage,dodge,usa
396,28.0,4,120.0,79.0,2625,18.6,82,1,ford ranger,ford,usa


## Idea #3: Change model year XX to XXXX representation

It seems the model year implicitly assumes the 20th century, and thus values are 19XX. Again, this is paranoid me, but I'd prefer to use the full representation, in case someone tries to use this model in the 21st century, thus making predictions more robust. Also, I find the use of whitespace in column names deeply offensive.

In [10]:
df_mfct_geo_year = df_mfct_geo.copy()
df_mfct_geo_year['model_year'] = df_mfct_geo_year['model year'].apply(lambda x: int(f"19{x}"))
del df_mfct_geo_year['model year']

In [11]:
df_mfct_geo_year

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,car name,mfct,geo,model_year
0,18.0,8,307.0,130.0,3504,12.0,1,chevrolet chevelle malibu,chevrolet,usa,1970
1,15.0,8,350.0,165.0,3693,11.5,1,buick skylark 320,buick,usa,1970
2,18.0,8,318.0,150.0,3436,11.0,1,plymouth satellite,plymouth,usa,1970
3,16.0,8,304.0,150.0,3433,12.0,1,amc rebel sst,amc,usa,1970
4,17.0,8,302.0,140.0,3449,10.5,1,ford torino,ford,usa,1970
...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,ford mustang gl,ford,usa,1982
394,44.0,4,97.0,52.0,2130,24.6,2,vw pickup,volkswagen,germany,1982
395,32.0,4,135.0,84.0,2295,11.6,1,dodge rampage,dodge,usa,1982
396,28.0,4,120.0,79.0,2625,18.6,1,ford ranger,ford,usa,1982


# Preparing the data

## Dealing with horsepower missing values

6 rows have a missing rows value. The rows are marked with '?', turning the whole column into a string column. We replace '?' with NaN, turn the column into floats, and use a linear interpolation provided by pandas. **Update: interpolating increases MAE, just drop the rows**

In [12]:
df_mfct_geo_year_hp = df_mfct_geo_year.copy()
# df_mfct_geo_year_hp['horsepower'] = df_mfct_geo_year_hp['horsepower'].apply(lambda x: int(x) if x != '?' else np.nan)
# df_mfct_geo_year_hp['horsepower'] = df_mfct_geo_year_hp['horsepower'].interpolate()

## Hot encoding the `geo` feature and the `mfct` feature

In [13]:
mfct_ohe = pd.get_dummies(df_mfct_geo_year['mfct'], dummy_na=True)
mfct_ohe = mfct_ohe.rename(columns={np.nan: 'mfct_nan'})
mfct_ohe

,amc,audi,bmw,buick,cadillac,chevrolet,chrysler,datsun,dodge,fiat,...,plymouth,pontiac,renault,saab,subaru,toyota,triumph,volkswagen,volvo,mfct_nan
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
395,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
geo_ohe = pd.get_dummies(df_mfct_geo_year['geo'], dummy_na=True)
geo_ohe = geo_ohe.rename(columns={np.nan: 'geo_nan'})
geo_ohe

,france,germany,italy,japan,sweden,uk,usa,geo_nan
0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...
393,0,0,0,0,0,0,1,0
394,0,1,0,0,0,0,0,0
395,0,0,0,0,0,0,1,0
396,0,0,0,0,0,0,1,0


# Putting it all together

Let's append the OHE columns together and drop the ones that they are replacing

In [15]:
df_concat = pd.concat([df_mfct_geo_year_hp, mfct_ohe, geo_ohe], axis=1)
df_concat

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,car name,mfct,geo,...,volvo,mfct_nan,france,germany,italy,japan,sweden,uk,usa,geo_nan
0,18.0,8,307.0,130.0,3504,12.0,1,chevrolet chevelle malibu,chevrolet,usa,...,0,0,0,0,0,0,0,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,1,buick skylark 320,buick,usa,...,0,0,0,0,0,0,0,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,1,plymouth satellite,plymouth,usa,...,0,0,0,0,0,0,0,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,1,amc rebel sst,amc,usa,...,0,0,0,0,0,0,0,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,1,ford torino,ford,usa,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,ford mustang gl,ford,usa,...,0,0,0,0,0,0,0,0,1,0
394,44.0,4,97.0,52.0,2130,24.6,2,vw pickup,volkswagen,germany,...,0,0,0,1,0,0,0,0,0,0
395,32.0,4,135.0,84.0,2295,11.6,1,dodge rampage,dodge,usa,...,0,0,0,0,0,0,0,0,1,0
396,28.0,4,120.0,79.0,2625,18.6,1,ford ranger,ford,usa,...,0,0,0,0,0,0,0,0,1,0


In [16]:
df_final = df_concat.drop(columns=['car name', 'mfct', 'geo'])
# for col in ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year']:
#     scaler = StandardScaler()
#     df_final[col] = scaler.fit_transform(df_final[col].values.reshape(-1, 1))
df_final

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,model_year,amc,audi,...,volvo,mfct_nan,france,germany,italy,japan,sweden,uk,usa,geo_nan
0,18.0,8,307.0,130.0,3504,12.0,1,1970,0,0,...,0,0,0,0,0,0,0,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,1,1970,0,0,...,0,0,0,0,0,0,0,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,1,1970,0,0,...,0,0,0,0,0,0,0,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,1,1970,1,0,...,0,0,0,0,0,0,0,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,1,1970,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,1,0
394,44.0,4,97.0,52.0,2130,24.6,2,1982,0,0,...,0,0,0,1,0,0,0,0,0,0
395,32.0,4,135.0,84.0,2295,11.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,1,0
396,28.0,4,120.0,79.0,2625,18.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,1,0


In [17]:
y, X = df_final[['mpg']].to_numpy().reshape(-1, ), df_final.drop(columns=['mpg']).to_numpy()
y.shape, X.shape

((397,), (397, 44))

# Model evaluation

We will use cross validation and the MEA to evaluate the model. We will test the effect of the hypotheses mentioned above regarding features. <p style="color: red; font-weight: bold">We should be careful about setting all random states in cross_validation and models to fixed values in order to get reproducible results.</p>

## First hypothesis - enhanced columns

In [18]:
model = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores = np.absolute(scores)
print('Mean MAE %.3f STD MAE %.3f' % (scores.mean(), scores.std()) )

/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/bratu/Desktop/dsp/venv/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a futur

Mean MAE 2.023 STD MAE 0.331


## Second hypothesis - plain columns

By plain columns we understand columns that do not carry any information about manufacturer or geolocation.

In [19]:
list(geo_ohe.columns)

['france', 'germany', 'italy', 'japan', 'sweden', 'uk', 'usa', 'geo_nan']

In [20]:
df_final_second_h = df_final.drop(columns=list(geo_ohe.columns) + list(mfct_ohe.columns))
df_final_second_h

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,model_year
0,18.0,8,307.0,130.0,3504,12.0,1,1970
1,15.0,8,350.0,165.0,3693,11.5,1,1970
2,18.0,8,318.0,150.0,3436,11.0,1,1970
3,16.0,8,304.0,150.0,3433,12.0,1,1970
4,17.0,8,302.0,140.0,3449,10.5,1,1970
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,1982
394,44.0,4,97.0,52.0,2130,24.6,2,1982
395,32.0,4,135.0,84.0,2295,11.6,1,1982
396,28.0,4,120.0,79.0,2625,18.6,1,1982


In [21]:
y_sec_hyp, X_sec_hyp = df_final_second_h[['mpg']].to_numpy().reshape(-1, ), df_final_second_h.drop(columns=['mpg']).to_numpy()
y_sec_hyp.shape, X_sec_hyp.shape

((397,), (397, 7))

In [22]:
model_sec_hyp = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv_sec_hyp = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores_sec_hyp = cross_val_score(model_sec_hyp, X_sec_hyp, y_sec_hyp, scoring='neg_mean_absolute_error', cv=cv_sec_hyp, n_jobs=-1)
scores_sec_hyp = np.absolute(scores_sec_hyp)
print('Mean MAE %.3f STD MAE %.3f' % (scores_sec_hyp.mean(), scores_sec_hyp.std()) )

Mean MAE 2.049 STD MAE 0.298


## Third hypothesis - plain columns minus origins 

Quick test to determine wether the origin column is relevant. It might encode all the geoencode information I was speculating above.

In [23]:
df_final_third_h = df_final.drop(columns=list(geo_ohe.columns) + list(mfct_ohe.columns) + ['origin'])
df_final_third_h

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,18.0,8,307.0,130.0,3504,12.0,1970
1,15.0,8,350.0,165.0,3693,11.5,1970
2,18.0,8,318.0,150.0,3436,11.0,1970
3,16.0,8,304.0,150.0,3433,12.0,1970
4,17.0,8,302.0,140.0,3449,10.5,1970
...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1982
394,44.0,4,97.0,52.0,2130,24.6,1982
395,32.0,4,135.0,84.0,2295,11.6,1982
396,28.0,4,120.0,79.0,2625,18.6,1982


In [24]:
y_third_hyp, X_third_hyp = df_final_third_h[['mpg']].to_numpy().reshape(-1, ), df_final_third_h.drop(columns=['mpg']).to_numpy()
y_third_hyp.shape, X_third_hyp.shape

((397,), (397, 6))

In [25]:
model_third_hyp = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv_third_hyp = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores_third_hyp = cross_val_score(model_third_hyp, X_sec_hyp, y_sec_hyp, scoring='neg_mean_absolute_error', cv=cv_third_hyp, n_jobs=-1)
scores_third_hyp = np.absolute(scores_third_hyp)
print('Mean MAE %.3f STD MAE %.3f' % (scores_third_hyp.mean(), scores_third_hyp.std()) )

Mean MAE 2.049 STD MAE 0.298


## Fourth hypothesis: Only the `manufacturer` enhanced column matters

In [26]:
df_final_fourth_h = df_final.drop(columns=list(geo_ohe.columns))
df_final_fourth_h

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,model_year,amc,audi,...,plymouth,pontiac,renault,saab,subaru,toyota,triumph,volkswagen,volvo,mfct_nan
0,18.0,8,307.0,130.0,3504,12.0,1,1970,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15.0,8,350.0,165.0,3693,11.5,1,1970,0,0,...,0,0,0,0,0,0,0,0,0,0
2,18.0,8,318.0,150.0,3436,11.0,1,1970,0,0,...,1,0,0,0,0,0,0,0,0,0
3,16.0,8,304.0,150.0,3433,12.0,1,1970,1,0,...,0,0,0,0,0,0,0,0,0,0
4,17.0,8,302.0,140.0,3449,10.5,1,1970,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,0,0
394,44.0,4,97.0,52.0,2130,24.6,2,1982,0,0,...,0,0,0,0,0,0,0,1,0,0
395,32.0,4,135.0,84.0,2295,11.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,0,0
396,28.0,4,120.0,79.0,2625,18.6,1,1982,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
y_fourth_h, X_fourth_h = df_final_fourth_h[['mpg']].to_numpy().reshape(-1, ), df_final_fourth_h.drop(columns=['mpg']).to_numpy()
y_fourth_h.shape, X_fourth_h.shape

((397,), (397, 36))

In [28]:
model_fourth_hyp = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv_fourth_hyp = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores_fourth_hyp = cross_val_score(model_fourth_hyp, X_fourth_h, y_fourth_h, scoring='neg_mean_absolute_error', cv=cv_fourth_hyp, n_jobs=-1)
scores_fourth_hyp = np.absolute(scores_fourth_hyp)
print('Mean MAE %.3f STD MAE %.3f' % (scores_fourth_hyp.mean(), scores_fourth_hyp.std()) )

Mean MAE 2.013 STD MAE 0.314


## Fifth hypothesis: Only the `geo` enhanced column matters

In [29]:
df_final_fifth_h = df_final.drop(columns=list(list(mfct_ohe.columns)))
df_final_fifth_h

,mpg,cylinders,displacement,horsepower,weight,acceleration,origin,model_year,france,germany,italy,japan,sweden,uk,usa,geo_nan
0,18.0,8,307.0,130.0,3504,12.0,1,1970,0,0,0,0,0,0,1,0
1,15.0,8,350.0,165.0,3693,11.5,1,1970,0,0,0,0,0,0,1,0
2,18.0,8,318.0,150.0,3436,11.0,1,1970,0,0,0,0,0,0,1,0
3,16.0,8,304.0,150.0,3433,12.0,1,1970,0,0,0,0,0,0,1,0
4,17.0,8,302.0,140.0,3449,10.5,1,1970,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,1,1982,0,0,0,0,0,0,1,0
394,44.0,4,97.0,52.0,2130,24.6,2,1982,0,1,0,0,0,0,0,0
395,32.0,4,135.0,84.0,2295,11.6,1,1982,0,0,0,0,0,0,1,0
396,28.0,4,120.0,79.0,2625,18.6,1,1982,0,0,0,0,0,0,1,0


In [30]:
y_fifth_h, X_fifth_h = df_final_fifth_h[['mpg']].to_numpy().reshape(-1, ), df_final_fifth_h.drop(columns=['mpg']).to_numpy()
y_fifth_h.shape, X_fifth_h.shape

((397,), (397, 15))

In [31]:
model_fifth_hyp = XGBRegressor(seed=42, random_state=42, colsample_bytree=1)
cv_fifth_hyp = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)
scores_fifth_hyp = cross_val_score(model_fifth_hyp, X_fifth_h, y_fifth_h, scoring='neg_mean_absolute_error', cv=cv_fifth_hyp, n_jobs=-1)
scores_fifth_hyp = np.absolute(scores_fifth_hyp)
print('Mean MAE %.3f STD MAE %.3f' % (scores_fifth_hyp.mean(), scores_fifth_hyp.std()) )

Mean MAE 2.021 STD MAE 0.303


# Conclusion

We conclude that our enhanced columns indeed improve the performance of the model (hypotheses #1 + #4). Adding only the manufacturer feature fares a better (#4) than adding both, although scores are quite close, and #1 has lower variation. These things considered, we will move with the engineered manufacturer feature into training, where we will leverage Bayesian search for hyperparameter optimization.

In [32]:
# with open('best.csv', 'w+') as fp:
#     df_final_fourth_h.to_csv(fp)